In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from ipywidgets import IntProgress
from IPython.display import display

In [10]:
!gdown --id '1osikvA4sCHiIilxVK-EAI0CppPsEtqSl' --output BloodData.csv

Downloading...
From: https://drive.google.com/uc?id=1osikvA4sCHiIilxVK-EAI0CppPsEtqSl
To: /content/BloodData.csv
100% 30.5k/30.5k [00:00<00:00, 4.53MB/s]


In [11]:
train = pd.read_csv("BloodData.csv")
train.head()

,Run,AGE,Bases,BioProject,BioSample,BMI,Bytes,HDRS,Library Name,Sample Name,sex,smoker,Source_material_ID,Patient,Clinical,Unidentified reads(%),Identified reads: cellular organisms(%),Identified reads: cellular organisms-bacteria(%),Kbp-bacteria,Terrabacteria group,Proteobacteria,Identified reads:virus(%),Kbp-virus,Identified reads:Eukaryota(%)
0,SRR10717942,33.9603,28486800,PRJNA596190,SAMN13614328,19.9,19880190,26,sample 107,dep_M0_6123456831,female,yes,sample 154,6123456831,dep_M3,1.89,98.07,98.05,27938,0,0,0.03,10.0,NaN
1,SRR10717943,50.0479,30235800,PRJNA596190,SAMN13614327,23.5,21152225,25,sample 106,dep_M0_6123456830,male,yes,sample 152,6123456830,dep_M0,21.03,78.94,78.78,23867,1,1,0.03,10.0,<0.01
2,SRR10717944,59.6277,31203600,PRJNA596190,SAMN13614326,25,21841668,23,sample 105,dep_M0_6123456829,female,no,sample 150,6123456829,dep_M3,1.04,98.94,98.89,30874,1,0,0.02,5.0,NaN
3,SRR10717945,33.8097,29617200,PRJNA596190,SAMN13614325,21.8,20861788,28,sample 104,dep_M0_6123456828,female,no,sample 148,6123456828,dep_M0,2.13,97.86,97.60,28981,1,1,0.01,2.0,0.01
4,SRR10717946,64.1780,25073400,PRJNA596190,SAMN13614324,24.3,17724087,25,sample 103,dep_M0_6123456827,male,no,sample 144,6123456827,dep_M3,5.03,94.92,94.85,23800,0,0,0.04,10.0,<0.01


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 24 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Run                                               168 non-null    object 
 1   AGE                                               168 non-null    float64
 2   Bases                                             168 non-null    int64  
 3   BioProject                                        168 non-null    object 
 4   BioSample                                         168 non-null    object 
 5   BMI                                               168 non-null    object 
 6   Bytes                                             168 non-null    int64  
 7   HDRS                                              168 non-null    object 
 8   Library Name                                      168 non-null    object 
 9   Sample Name          

In [13]:
train["AGE"].value_counts()

45.0    4
51.0    3
55.0    3
54.0    3
40.0    3
       ..
49.0    1
36.0    1
52.0    1
24.0    1
23.0    1
Name: AGE, Length: 89, dtype: int64

In [14]:
#BMI補值 補平均數
print(train["BMI"].value_counts())
total_BMI = 0
count = 0
for i in range(len(train["BMI"])):
  if train["BMI"][i] != 'missing':
    total_BMI += float(train["BMI"][i])
    count += 1
average_BMI = round(total_BMI/count,3)
print(total_BMI)
print(average_BMI)
for i in range(len(train["BMI"])):
  if train["BMI"][i] == 'missing':
    train["BMI"][i] = str(average_BMI)
print(train["BMI"].value_counts())

missing    59
25          3
22.1        3
21.8        3
24.3        3
           ..
27.6        1
22.6        1
32          1
33.1        1
22.3        1
Name: BMI, Length: 79, dtype: int64
2707.0999999999995
24.836
24.836    59
25         3
22.1       3
21.8       3
24.3       3
          ..
22.6       1
32         1
33.1       1
18.3       1
22.3       1
Name: BMI, Length: 79, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
#HDRS補值
#健康的補0、Dep_M0、Dep_M3補各自的平均值
print(train["HDRS"].value_counts())
total_HDRS_M0 = 0
total_HDRS_M3 = 0
count_HDRS_M0 = 0
count_HDRS_M3 = 0
for i in range(len(train["HDRS"])):
  if train["HDRS"][i] != 'missing':
    if train['Clinical'][i]=='dep_M0':
      total_HDRS_M0 += float(train["HDRS"][i])
      count_HDRS_M0 += 1
    if train["HDRS"][i] != 'missing':
      total_HDRS_M3 += float(train["HDRS"][i])
      count_HDRS_M3 += 1
average_HDRS_M0 = math.floor(total_HDRS_M0/count_HDRS_M0)
average_HDRS_M3 = math.floor(total_HDRS_M3/count_HDRS_M3)
print('The average HDRS before treated is ',average_HDRS_M0)
print('The average HDRS after treated is ',average_HDRS_M3)
for i in range(len(train["HDRS"])):
  if train["HDRS"][i] == 'missing':
    if train['Clinical'][i]=='dep_M0':
      train["HDRS"][i] = str(average_HDRS_M0)
    if train['Clinical'][i]=='dep_M3':
      train["HDRS"][i] = str(average_HDRS_M3)
    if train['Clinical'][i]=='control':
      train["HDRS"][i] = '0'
print(train["HDRS"].value_counts())

missing    56
18         10
22          8
24          7
26          7
28          7
23          6
25          6
9           6
20          5
21          5
19          4
5           4
7           4
11          3
3           3
10          3
6           2
12          2
2           2
35          2
4           2
1           2
8           2
29          2
0           1
34          1
27          1
30          1
14          1
15          1
13          1
37          1
Name: HDRS, dtype: int64
The average HDRS before treated is  18
The average HDRS after treated is  17
0     52
18    13
22     8
28     7
26     7
24     7
9      6
23     6
25     6
20     5
21     5
19     4
5      4
7      4
3      3
11     3
10     3
4      2
35     2
12     2
2      2
6      2
1      2
8      2
17     2
29     2
34     1
27     1
30     1
14     1
15     1
13     1
37     1
Name: HDRS, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [16]:
#smoker yes/no 統一改成小寫 3個missing sample補眾數'no'
print(train["smoker"].value_counts())
for i in range(len(train["smoker"])):
  if train["smoker"][i] == 'Yes':
    train["smoker"][i] = 'yes'
  if train["smoker"][i] == 'No':
    train["smoker"][i] = 'no'
  if train["smoker"][i] == 'missing':
    train["smoker"][i] = 'no'
print(train["smoker"].value_counts())

no         79
No         51
yes        30
Yes         5
missing     3
Name: smoker, dtype: int64
no     133
yes     35
Name: smoker, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [17]:
#剩下的空值補0
train["Proteobacteria"].fillna(0,inplace=True)
train["Identified reads:Eukaryota(%)"].fillna(0,inplace=True)
train['Kbp-virus'].fillna(0,inplace=True)

In [18]:
for i in range(len(train["Identified reads:Eukaryota(%)"])):
  if train["Identified reads:Eukaryota(%)"][i] == '<0.01':
    train["Identified reads:Eukaryota(%)"][i] = '0.005'
  if train["Identified reads:virus(%)"][i] == '<0.01':
    train["Identified reads:virus(%)"][i] = '0.005'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 24 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Run                                               168 non-null    object 
 1   AGE                                               168 non-null    float64
 2   Bases                                             168 non-null    int64  
 3   BioProject                                        168 non-null    object 
 4   BioSample                                         168 non-null    object 
 5   BMI                                               168 non-null    object 
 6   Bytes                                             168 non-null    int64  
 7   HDRS                                              168 non-null    object 
 8   Library Name                                      168 non-null    object 
 9   Sample Name          

In [20]:
CAT_COL = ["Run", "Bases", "BioProject", "BioSample", "Library Name",
          "Sample Name", "Patient", "sex", "smoker", "Source_material_ID",
          "Clinical", "Terrabacteria group", "Proteobacteria"]

NUM_COL = ["AGE", "BMI", "Bytes", "HDRS", "Unidentified reads(%)", "Identified reads: cellular organisms(%)",
          "Identified reads: cellular organisms-bacteria(%)", "Kbp-bacteria",
          "Identified reads:virus(%)", "Kbp-virus", "Identified reads:Eukaryota(%)"]
cat_col = []
num_col = []
for col in train:
  if col in CAT_COL:
    cat_col.append(col)
  elif col in NUM_COL:
    num_col.append(col)
for col in cat_col:
  train[col] = train[col].astype(str)

df_cat = train.loc[:,cat_col] # take all the categorical columns
df_cat = pd.get_dummies(df_cat) # one hot encoding

df_num = train.loc[:,num_col] # take all the numerical columns
df_final = pd.concat([df_cat, df_num], axis=1) # concat categorical/numerical data

In [21]:
df_final.head()

,Run_SRR10717942,Run_SRR10717943,Run_SRR10717944,Run_SRR10717945,Run_SRR10717946,Run_SRR10717947,Run_SRR10717948,Run_SRR10717949,Run_SRR10717950,Run_SRR10717951,Run_SRR10717952,Run_SRR10717953,Run_SRR10717954,Run_SRR10717955,Run_SRR10717956,Run_SRR10717957,Run_SRR10717958,Run_SRR10717959,Run_SRR10717960,Run_SRR10717961,Run_SRR10717962,Run_SRR10717963,Run_SRR10717964,Run_SRR10717965,Run_SRR10717966,Run_SRR10717967,Run_SRR10717968,Run_SRR10717969,Run_SRR10717970,Run_SRR10717971,Run_SRR10717972,Run_SRR10717973,Run_SRR10717974,Run_SRR10717975,Run_SRR10717976,Run_SRR10717977,Run_SRR10717978,Run_SRR10717979,Run_SRR10717980,Run_SRR10717981,...,Patient_6123456822,Patient_6123456823,Patient_6123456824,Patient_6123456825,Patient_6123456826,Patient_6123456827,Patient_6123456828,Patient_6123456829,Patient_6123456830,Patient_6123456831,Patient_6123456832,Patient_6123456833,Patient_6123456834,Patient_6123456835,Patient_6123456836,Patient_6123456837,Patient_6123456838,Patient_6123456839,Patient_6123456840,Patient_6123456841,Patient_6123456842,Patient_6123456843,Clinical_control,Clinical_dep_M0,Clinical_dep_M3,Terrabacteria group_0,Terrabacteria group_1,Proteobacteria_0,Proteobacteria_1,AGE,BMI,Bytes,HDRS,Unidentified reads(%),Identified reads: cellular organisms(%),Identified reads: cellular organisms-bacteria(%),Kbp-bacteria,Identified reads:virus(%),Kbp-virus,Identified reads:Eukaryota(%)
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,33.9603,19.9,19880190,26,1.89,98.07,98.05,27938,0.03,10.0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,50.0479,23.5,21152225,25,21.03,78.94,78.78,23867,0.03,10.0,0.005
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,59.6277,25,21841668,23,1.04,98.94,98.89,30874,0.02,5.0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,33.8097,21.8,20861788,28,2.13,97.86,97.60,28981,0.01,2.0,0.01
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,64.1780,24.3,17724087,25,5.03,94.92,94.85,23800,0.04,10.0,0.005


In [22]:
not_select = ["Run", "Bases", "BioProject", "BioSample", "Library Name", "Sample Name", "Bytes", "Patient", "Source_material_ID"]
train_select = train.drop(not_select,axis=1)
cat_col = []
num_col = []
for col in train_select:
  if col in CAT_COL:
    cat_col.append(col)
  elif col in NUM_COL:
    num_col.append(col)

for col in cat_col:
  if train_select[col].dtype != "O":
    # print(col)
    train_select[col] = train_select[col].astype(str)

df_cat_select = train_select.loc[:,cat_col] # take all the categorical columns
df_cat_select = pd.get_dummies(df_cat_select) # one hot encoding

df_num_select = train_select.loc[:,num_col] # take all the numerical columns
df_final_select = pd.concat([df_cat_select, df_num_select], axis=1) # concat categorical/numerical data

In [23]:
df_final_select.head()

,sex_female,sex_male,smoker_no,smoker_yes,Clinical_control,Clinical_dep_M0,Clinical_dep_M3,Terrabacteria group_0,Terrabacteria group_1,Proteobacteria_0,Proteobacteria_1,AGE,BMI,HDRS,Unidentified reads(%),Identified reads: cellular organisms(%),Identified reads: cellular organisms-bacteria(%),Kbp-bacteria,Identified reads:virus(%),Kbp-virus,Identified reads:Eukaryota(%)
0,1,0,0,1,0,0,1,1,0,1,0,33.9603,19.9,26,1.89,98.07,98.05,27938,0.03,10.0,0
1,0,1,0,1,0,1,0,0,1,0,1,50.0479,23.5,25,21.03,78.94,78.78,23867,0.03,10.0,0.005
2,1,0,1,0,0,0,1,0,1,1,0,59.6277,25,23,1.04,98.94,98.89,30874,0.02,5.0,0
3,1,0,1,0,0,1,0,0,1,0,1,33.8097,21.8,28,2.13,97.86,97.60,28981,0.01,2.0,0.01
4,0,1,1,0,0,0,1,1,0,1,0,64.1780,24.3,25,5.03,94.92,94.85,23800,0.04,10.0,0.005


In [ ]:
df_final_select_num = df_final_select.values
for i in range(len(df_final_select_num)):
  for j in range(len(df_final_select_num[0])):
    df_final_select_num[i][j] = float(df_final_select_num[i][j])
correlation_matrix = np.corrcoef(df_final_select_num)
print(correlation_matrix)

AttributeError: ignored

In [ ]:
print(len(df_final_select_num[1]))

21


In [24]:
not_select = ["Run", "Bases", "BioProject", "BioSample", "Library Name", "Sample Name", "Bytes", "Clinical", "Patient", "Source_material_ID"]
train_select = train.drop(not_select,axis=1)
train_select.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 14 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   AGE                                               168 non-null    float64
 1   BMI                                               168 non-null    object 
 2   HDRS                                              168 non-null    object 
 3   sex                                               168 non-null    object 
 4   smoker                                            168 non-null    object 
 5   Unidentified reads(%)                             168 non-null    float64
 6   Identified reads: cellular organisms(%)           168 non-null    float64
 7   Identified reads: cellular organisms-bacteria(%)  168 non-null    float64
 8   Kbp-bacteria                                      168 non-null    int64  
 9   Terrabacteria group  

In [25]:
cat_col = []
num_col = []
for col in train_select:
  if col in CAT_COL:
    cat_col.append(col)
  elif col in NUM_COL:
    num_col.append(col)

for col in cat_col:
  if train_select[col].dtype != "O":
    # print(col)
    train_select[col] = train_select[col].astype(str)

df_cat_select = train_select.loc[:,cat_col] # take all the categorical columns
df_cat_select = pd.get_dummies(df_cat_select) # one hot encoding

df_num_select = train_select.loc[:,num_col] # take all the numerical columns
df_final_select = pd.concat([df_cat_select, df_num_select], axis=1) # concat categorical/numerical data

In [26]:
df_final_select.head()

,sex_female,sex_male,smoker_no,smoker_yes,Terrabacteria group_0,Terrabacteria group_1,Proteobacteria_0,Proteobacteria_1,AGE,BMI,HDRS,Unidentified reads(%),Identified reads: cellular organisms(%),Identified reads: cellular organisms-bacteria(%),Kbp-bacteria,Identified reads:virus(%),Kbp-virus,Identified reads:Eukaryota(%)
0,1,0,0,1,1,0,1,0,33.9603,19.9,26,1.89,98.07,98.05,27938,0.03,10.0,0
1,0,1,0,1,0,1,0,1,50.0479,23.5,25,21.03,78.94,78.78,23867,0.03,10.0,0.005
2,1,0,1,0,0,1,1,0,59.6277,25,23,1.04,98.94,98.89,30874,0.02,5.0,0
3,1,0,1,0,0,1,0,1,33.8097,21.8,28,2.13,97.86,97.60,28981,0.01,2.0,0.01
4,0,1,1,0,1,0,1,0,64.1780,24.3,25,5.03,94.92,94.85,23800,0.04,10.0,0.005


#Random Forest Classifier

In [27]:
#Use RandomForestClassifier to predict Clinical
x = df_final_select
y = train["Clinical"]
# y = np.array(y,dtype=int)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.1,random_state=0)

#RandomForest
rfc = RandomForestClassifier()
#rfc=RandomForestClassifier(n_estimators=100,n_jobs = -1,random_state =50, min_samples_leaf = 10)
rfc.fit(X_train,y_train)
y_predict = rfc.predict(X_train)
score_rfc = rfc.score(X_test,y_test)
print("Random Forest Accuracy = ",score_rfc*100," %")

Random Forest Accuracy =  76.47058823529412  %


In [28]:
from sklearn.model_selection import KFold
x = df_final_select
y = train["Clinical"]
kf = KFold(n_splits=5)
best_accuracy = 0
for train_index , test_index in kf.split(x):
    X_train, X_test, y_train, y_test = x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    rfc = RandomForestClassifier()
    rfc.fit(X_train,y_train)
    y_predict = rfc.predict(X_train)
    accuracy = rfc.score(X_test,y_test)
    print("Random Forest Accuracy = ",accuracy*100,"%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_rfc = rfc
print("Best Accuracy = ",best_accuracy*100,"%")

Random Forest Accuracy =  73.52941176470588 %
Random Forest Accuracy =  70.58823529411765 %
Random Forest Accuracy =  82.35294117647058 %
Random Forest Accuracy =  57.57575757575758 %
Random Forest Accuracy =  60.60606060606061 %
Best Accuracy =  82.35294117647058 %


In [29]:
#random forest final accuracy
x_train_new = x
y_train_new = train["Clinical"]
y_train_new = y_train_new.reset_index(drop=True)

y_pred = best_rfc.predict(x_train_new)
count = 0

for i in range(y_pred.shape[0]):
    if y_pred[i] != y_train_new[i]:
        count += 1
    print(y_pred[i],y_train_new[i])
rfc_accuracy = 1-count/y_pred.shape[0]
print("Random Forest Accuracy = ",rfc_accuracy*100,"%")

dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
control control
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
control control
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
control control
dep_M3 dep_M3
dep_M0 dep_M0
control control
control control
control control
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
control control
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
control control
control control
control control
control control
control control
control control
control control
control control
control control
control control
control control
control control
control control
control control
control control
cont

#SVM

In [30]:
from sklearn import svm
#Use SVM to predict Clinical
x = df_final_select
y = train["Clinical"]
# y = np.array(y,dtype=int)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.1,random_state=0)

clf = svm.SVC()
clf.fit(X_train,y_train)
y_predict = clf.predict(X_train)
score_clf = clf.score(X_test,y_test)
print("SVM Accuracy = ",score_clf*100," %")

SVM Accuracy =  35.294117647058826  %


In [31]:
from sklearn.model_selection import KFold
x = df_final_select
y = train["Clinical"]
kf = KFold(n_splits=5)
best_accuracy = 0
for train_index , test_index in kf.split(x):
    X_train, X_test, y_train, y_test = x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    clf = svm.SVC()
    clf.fit(X_train,y_train)
    y_predict = clf.predict(X_train)
    accuracy = clf.score(X_test,y_test)
    print("SVM Accuracy = ",accuracy*100,"%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_clf = clf
print("Best Accuracy = ",best_accuracy*100,"%")

SVM Accuracy =  8.823529411764707 %
SVM Accuracy =  20.588235294117645 %
SVM Accuracy =  11.76470588235294 %
SVM Accuracy =  21.21212121212121 %
SVM Accuracy =  15.151515151515152 %
Best Accuracy =  21.21212121212121 %


In [32]:
#SVM final accuracy
x_train_new = x
y_train_new = train["Clinical"]
y_train_new = y_train_new.reset_index(drop=True)

y_pred = best_clf.predict(x_train_new)
count = 0

for i in range(y_pred.shape[0]):
    print(y_pred[i],y_train_new[i])
    if y_pred[i] != y_train_new[i]:
        count += 1
clf_accuracy = 1-count/y_pred.shape[0]
print("SVM Accuracy = ",clf_accuracy*100,"%")

control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
dep_M0 dep_M3
control dep_M0
control dep_M3
control dep_M0
control control
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control control
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control control
control dep_M3
control dep_M0
control control
control control
control control
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control control
control dep_M0
control dep_M3
control dep_M0
control dep_M3
dep_M0 dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control control
control control
control control
control control
control control
dep_M0 control
control control
control control
control control
control control
control control
control control
control c

#Neural network MLPClassifier

In [33]:
from sklearn.neural_network import MLPClassifier
#Use Neural Network MLPClassifier to predict Clinical
x = df_final_select
y = train["Clinical"]
# y = np.array(y,dtype=int)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.1,random_state=0)

nnclf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(50, 30), random_state=1, max_iter=2000)
nnclf.fit(X_train,y_train)
y_predict = nnclf.predict(X_train)
score_nnclf = nnclf.score(X_test,y_test)
print("Neural Network Accuracy = ",score_nnclf*100," %")

Neural Network Accuracy =  35.294117647058826  %


In [34]:
from sklearn.model_selection import KFold
x = df_final_select
y = train["Clinical"]
kf = KFold(n_splits=5)
best_accuracy = 0
for train_index , test_index in kf.split(x):
    X_train, X_test, y_train, y_test = x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    nnclf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(50, 30), random_state=1, max_iter=2000)
    nnclf.fit(X_train,y_train)
    y_predict = nnclf.predict(X_train)
    accuracy = nnclf.score(X_test,y_test)
    print("NN Accuracy = ",accuracy*100,"%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_nnclf = nnclf
print("NN Accuracy = ",best_accuracy*100,"%")

NN Accuracy =  44.11764705882353 %
NN Accuracy =  55.88235294117647 %
NN Accuracy =  70.58823529411765 %
NN Accuracy =  42.42424242424242 %
NN Accuracy =  12.121212121212121 %
NN Accuracy =  70.58823529411765 %


In [35]:
#NN final accuracy
x_train_new = x
y_train_new = train["Clinical"]
y_train_new = y_train_new.reset_index(drop=True)

y_pred = best_nnclf.predict(x_train_new)
count = 0

for i in range(y_pred.shape[0]):
    print(y_pred[i],y_train_new[i])
    if y_pred[i] != y_train_new[i]:
        count += 1
nnclf_accuracy = 1-count/y_pred.shape[0]
print("NN Accuracy = ",nnclf_accuracy*100,"%")

control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control control
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control control
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control control
control dep_M3
control dep_M0
control control
control control
control control
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control control
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control dep_M3
control dep_M0
control control
control control
control control
control control
control control
dep_M3 control
control control
control control
control control
control control
control control
control control
control

#Logistic Regression

In [41]:
from sklearn.linear_model import LogisticRegression
#Use Logistic Regression to predict Clinical
x = df_final_select
y = train["Clinical"]
# y = np.array(y,dtype=int)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.1,random_state=0)

logclf = LogisticRegression(random_state=0).fit(X_train,y_train)
logclf.predict(X_train)
logclf.predict_proba(X_train)
score_logclf = logclf.score(X_test,y_test)
print("Logistic Regression Accuracy = ",score_logclf*100," %")

Logistic Regression Accuracy =  70.58823529411765  %


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [42]:
from sklearn.model_selection import KFold
x = df_final_select
y = train["Clinical"]
kf = KFold(n_splits=5)
best_accuracy = 0
for train_index , test_index in kf.split(x):
    X_train, X_test, y_train, y_test = x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    logclf = LogisticRegression(random_state=0).fit(X_train,y_train)
    y_predict = logclf.predict(X_train)
    accuracy = logclf.score(X_test,y_test)
    print("Logistic Regression Accuracy = ",accuracy*100,"%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_logclf = logclf
print("Logistic Regression Accuracy = ",best_accuracy*100,"%")

Logistic Regression Accuracy =  61.76470588235294 %
Logistic Regression Accuracy =  67.64705882352942 %
Logistic Regression Accuracy =  85.29411764705883 %
Logistic Regression Accuracy =  45.45454545454545 %
Logistic Regression Accuracy =  60.60606060606061 %
Logistic Regression Accuracy =  85.29411764705883 %


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [43]:
#Logistic final accuracy
x_train_new = x
y_train_new = train["Clinical"]
y_train_new = y_train_new.reset_index(drop=True)

y_pred = best_logclf.predict(x_train_new)
count = 0

for i in range(y_pred.shape[0]):
    print(y_pred[i],y_train_new[i])
    if y_pred[i] != y_train_new[i]:
        count += 1
logclf_accuracy = 1-count/y_pred.shape[0]
print("Logistic Regression Accuracy = ",logclf_accuracy*100,"%")

dep_M0 dep_M3
dep_M3 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M0 dep_M3
dep_M3 dep_M0
dep_M0 dep_M3
dep_M0 dep_M0
dep_M0 dep_M3
dep_M3 dep_M0
control control
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
control control
dep_M3 dep_M3
dep_M3 dep_M0
dep_M3 dep_M3
dep_M3 dep_M0
dep_M3 dep_M3
dep_M3 dep_M0
dep_M0 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M3 dep_M0
control control
dep_M3 dep_M3
dep_M3 dep_M0
dep_M3 control
dep_M0 control
dep_M3 control
dep_M0 dep_M3
dep_M3 dep_M0
dep_M0 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
control control
dep_M3 dep_M0
dep_M0 dep_M3
dep_M0 dep_M0
dep_M3 dep_M3
dep_M0 dep_M0
dep_M0 dep_M3
dep_M3 dep_M0
dep_M0 dep_M3
dep_M0 dep_M0
control control
control control
control control
control control
control control
dep_M0 control
control control
control control
control control
control control
control control
control control
control control
control control
control control
control 

In [44]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(135, 18) (135,)
(33, 18) (33,)
